In [3]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit #for data preprocessing and crass validating 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LogisticRegression #logistic Regression
from sklearn.ensemble import RandomForestRegressor #Random Forest 

from statistics import mean
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval #for hyperparameter tuning and minimizing

from cyclic_boosting.pipelines import pipeline_CBClassifier
from sklearn.ensemble import HistGradientBoostingRegressor

from datetime import date
from datetime import datetime

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping

import itertools
from keras.layers import LSTM

In [10]:
samples = pd.read_csv('preprocessed.csv')
samples['date'] = pd.to_datetime(samples['date'])
reading_types = pd.read_csv('reading_types.csv')

samples.info()

df_lst = [(k, v) for k, v in samples.groupby('value_type_id')]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957826 entries, 0 to 957825
Data columns (total 40 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Unnamed: 0     957826 non-null  int64         
 1   date           957826 non-null  datetime64[ns]
 2   value_type_id  957826 non-null  int64         
 3   value          957826 non-null  float64       
 4   work_hours     957826 non-null  bool          
 5   Fall           957826 non-null  bool          
 6   Spring         957826 non-null  bool          
 7   Summer         957826 non-null  bool          
 8   Winter         957826 non-null  bool          
 9   day type       957826 non-null  int64         
 10  trimester_day  957826 non-null  int64         
 11  building_1     957826 non-null  bool          
 12  building_2     957826 non-null  bool          
 13  building_3     957826 non-null  bool          
 14  building_6     957826 non-null  bool          
 15  

In [20]:
pred: pd.DataFrame
for num, df in df_lst[0:1]:
    df = df.drop(columns=['value_type_id', 'Unnamed: 0'], axis = 1)
    df = df.sort_values(by='date')  
    sz = len(df)
    partition = sz // 5
    df = df.rename({'date' : 'ds'}, axis = 1)
    df = df.rename({'value' : 'y'}, axis = 1)
    train = df[:4*partition] 
    test_x = df[4*partition:].drop('y', axis = 1)
    test_y = df[4*partition:]['y'] 
    print(reading_types.at[num - 1, 'reading_type_name'], '-'*80)
    # x_train, x_test, y_train, y_test = train_test_split(df.drop('y', axis = 1), df['y'])
    # tst_cv = TimeSeriesSplit(n_splits=5, gap = 48)

    model = Prophet()
    model.fit(train)

    predictions = model.predict(test_x)
    print(predictions.head(100))
    print(test_y.head(100))
    yhat = predictions['yhat']
    print(r2_score(test_y, yhat))

    

CO2 --------------------------------------------------------------------------------


23:02:27 - cmdstanpy - INFO - Chain [1] start processing
23:03:07 - cmdstanpy - INFO - Chain [1] done processing


                    ds       trend  yhat_lower  yhat_upper  trend_lower  \
0  2023-10-03 05:36:00  501.929331  402.945826  618.496633   501.929331   
1  2023-10-03 05:44:00  501.929347  402.326871  614.587238   501.929347   
2  2023-10-03 05:52:00  501.929364  400.887608  621.443491   501.929364   
3  2023-10-03 05:55:00  501.929371  403.475454  615.541101   501.929371   
4  2023-10-03 06:04:00  501.929390  400.201332  619.282254   501.929390   
..                 ...         ...         ...         ...          ...   
95 2023-10-03 12:46:00  501.930237  427.396902  636.114958   501.930237   
96 2023-10-03 12:47:00  501.930240  420.853691  649.384940   501.930240   
97 2023-10-03 12:51:00  501.930248  418.427656  641.188021   501.930248   
98 2023-10-03 12:52:00  501.930250  424.447407  639.199608   501.930250   
99 2023-10-03 13:11:00  501.930290  421.036145  651.683045   501.930290   

    trend_upper  additive_terms  additive_terms_lower  additive_terms_upper  \
0    501.929331     